# 벡터 클래스에 단항 연산자 추가 (-,+)

In [59]:

from array import array
import math
import reprlib
import functools
import operator
import itertools
import numbers

class Vector :
    typecode ='d'

    # 시퀀스형 객체를 입력받아서 Vector를 생성하기.
    
    def __init__(self, components) :
        self._components = array(self.typecode, components)
    
    def __iter__(self) :
        return iter(self._components)
    
    def __len__(self) :
        return len(self._components)
    
    def __pos__(self) :
        return Vector(self)
    
    def __neg__(self) :
        return Vector(-x for x in self)
        
    def __repr__(self) :
        # reprlib 은 components 를 제한된 길이로 출력한다.
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f"Vector({components})"
    
    shortcut_names = 'xyzt'
    
    def __getattr__(self, name) :
        cls = type(self)
        if len(name) == 1 :
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components) : 
                return self._components[pos]
        msg = f"{cls.__name__} object has no attribute {name}"
        raise AttributeError(msg) 
        
    # 생성자와 같은 기타 모든 속성을 할당할 때 사용되므로 주의하자
    def __setattr__(self, name, value) :
        cls = type(self)
        if len(name) == 1 :
            if name in cls.shortcut_names :
                error = f'readonly attribute {name}'
            elif name.islower() :
                error = f"can't set attributes 'a' to 'z' in {cls.__name__}"
            else :
                error = ''
            if error :
                raise AttributeError(error)
            
        # super()은 메타 Vector ..?? 
        super().__setattr__(name, value)
    
    def __str__(self) :
        return str(tuple(self))
    
    def __bytes__(self) :
        return bytes([ord(self.typecode)]) + bytes(self._components)
    
    def __eq__(self, other) :
        if isinstance(other, Vector) :
            return (len(self) == len(other) and
                    all(a == b for a,b, in zip(self, other)))
        else :
            return NotImplemented
        
        
    def __hash__(self) :
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)
        
    def __add__(self, other) :
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a+b for a,b in pairs)
    
    def __radd__(self, other) :
        return self + other
    
    # 유클리드 거리
    def __abs__(self) :
        return math.sqrt(sum(x * x for x in self))
    
    def __mul__(self, scalar) :
        if isinstance(scalar, numbers.Real) :
            return Vector(x * scalar for x in self)
        else : 
            return NotImplemented
    
    def __rmul__(self, scalar) :
        return self * scalar
    
    # abs가 0이면 False 나머지는 True
    def __bool__(self) :
        return bool(abs(self))
    
    def __complex__(self) :
        return complex(self.x, self.y)
    
    #classmethod는 첫번째 함수로 자신의 클래스를 받는다.
    @classmethod
    def frombytes(cls, octets) :
        typecode = chr(octets[0])
        # memoryview 는 구조체를 복사하지 않고 메모리를 공유할 수 있게 해준다.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [60]:
Vector([3,4,6]) + Vector([4,5])

Vector([7.0, 9.0, 6.0])

In [61]:
Vector([1,2,3]) + (10,20)

Vector([11.0, 22.0, 3.0])

In [62]:
(10,20) + Vector([1,2,3])

Vector([11.0, 22.0, 3.0])

In [63]:
Vector([1,2,3]) * 4

Vector([4.0, 8.0, 12.0])

In [64]:
4 * Vector([1,2,3])

Vector([4.0, 8.0, 12.0])